# 📚 Dental Books Scraper & Data Processing (FAST VERSION)

This notebook:
1. Mounts Google Drive to access your dental books folder
2. Scrapes ALL dental books from the library website (all categories) **with parallel downloads**
3. Processes all PDFs for the RAFT pipeline

**Google Drive Folder:** MyDentalBooks

**Library URL:** http://43.230.198.52/lib/book/

**Speed:** ~10x faster with parallel downloads (10 concurrent connections)

## 1. Setup & Mount Google Drive

In [ ]:
# Install dependencies
!pip install -q requests beautifulsoup4 tqdm PyMuPDF lxml

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Your dental books folder
DRIVE_FOLDER = "/content/drive/MyDrive/MyDentalBooks"

# Verify folder exists
import os
if os.path.exists(DRIVE_FOLDER):
    existing_books = [f for f in os.listdir(DRIVE_FOLDER) if f.lower().endswith('.pdf')]
    print(f"✓ Found MyDentalBooks folder with {len(existing_books)} existing PDF files")
else:
    os.makedirs(DRIVE_FOLDER, exist_ok=True)
    existing_books = []
    print(f"Created folder: {DRIVE_FOLDER}")

## 2. Fast Parallel Scraper

In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse, unquote, quote
from pathlib import Path
import re
import time
from tqdm.notebook import tqdm
from dataclasses import dataclass
from typing import List, Dict, Tuple
from concurrent.futures import ThreadPoolExecutor, as_completed
import threading

@dataclass
class BookInfo:
    """Information about a book."""
    title: str
    url: str
    category: str
    filename: str


class FastDentalLibraryScraper:
    """
    Fast parallel scraper for the dental library.
    Uses multiple threads for concurrent downloads.
    """
    
    BASE_URL = "http://43.230.198.52/lib/book/"
    
    # All known categories
    CATEGORIES = [
        "Biochemistry", "Cancer", "Conservative Dentistry",
        "Dental Anatomy & Oral Histology", "Dental Materials",
        "Dermatology", "Dictionary", "Forensic",
        "General Medicine", "General Pathology", "General Surgery",
        "Genetics", "Hematology", "Human Anatomy", "Human Physiology",
        "Immunology", "Implant Dentistry", "Microbiology", "Miscellaneous",
        "Oral & Dental Anatomy", "Oral Medicine & Radiology",
        "Oral Pathology", "Oral Surgery", "Orthodontics",
        "Pedodontics", "Periodontics", "Pharmacology",
        "Prosthodontics", "Public Health Dentistry",
    ]
    
    def __init__(self, output_dir: str, max_workers: int = 10, timeout: int = 60):
        """
        Initialize fast scraper.
        
        Args:
            output_dir: Directory to save downloaded books
            max_workers: Number of parallel download threads (default: 10)
            timeout: Request timeout in seconds
        """
        self.output_dir = Path(output_dir)
        self.output_dir.mkdir(parents=True, exist_ok=True)
        self.max_workers = max_workers
        self.timeout = timeout
        
        # Thread-local sessions for connection reuse
        self._local = threading.local()
        
        # Statistics (thread-safe)
        self._lock = threading.Lock()
        self.stats = {
            'discovered': 0,
            'downloaded': 0,
            'already_exists': 0,
            'failed': 0
        }
    
    def _get_session(self) -> requests.Session:
        """Get thread-local session."""
        if not hasattr(self._local, 'session'):
            session = requests.Session()
            session.headers.update({
                "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36",
                "Accept": "*/*",
            })
            # Connection pooling for speed
            adapter = requests.adapters.HTTPAdapter(
                pool_connections=20,
                pool_maxsize=20,
                max_retries=3
            )
            session.mount('http://', adapter)
            self._local.session = session
        return self._local.session
    
    def get_page(self, url: str) -> BeautifulSoup:
        """Fetch and parse a webpage."""
        try:
            response = self._get_session().get(url, timeout=30)
            response.raise_for_status()
            return BeautifulSoup(response.content, "html.parser")
        except Exception as e:
            print(f"Error fetching {url}: {e}")
            return None
    
    def discover_categories(self) -> List[str]:
        """Discover all category folders."""
        print(f"Discovering categories from {self.BASE_URL}")
        
        soup = self.get_page(self.BASE_URL)
        if soup is None:
            print("Using predefined categories")
            return self.CATEGORIES
        
        categories = []
        for link in soup.find_all("a", href=True):
            href = link["href"]
            if href in ["../", "/", "#"] or href.startswith("?"):
                continue
            if href.endswith("/"):
                category_name = unquote(href.rstrip("/"))
                if category_name and not category_name.startswith("."):
                    categories.append(category_name)
        
        if categories:
            print(f"Found {len(categories)} categories")
            return categories
        return self.CATEGORIES
    
    def discover_books_in_category(self, category: str) -> List[BookInfo]:
        """Discover all PDF books in a category."""
        books = []
        category_url = urljoin(self.BASE_URL, quote(category) + "/")
        
        soup = self.get_page(category_url)
        if soup is None:
            return books
        
        for link in soup.find_all("a", href=True):
            href = link["href"]
            if href.lower().endswith(".pdf"):
                full_url = urljoin(category_url, href)
                filename = unquote(href)
                title = self._clean_title(Path(filename).stem)
                
                books.append(BookInfo(
                    title=title,
                    url=full_url,
                    category=category,
                    filename=filename
                ))
        
        return books
    
    def discover_all_books(self, categories: List[str] = None) -> List[BookInfo]:
        """Discover all books from all categories (parallel)."""
        if categories is None:
            categories = self.discover_categories()
        
        all_books = []
        print(f"\nScanning {len(categories)} categories in parallel...\n")
        
        # Parallel category scanning
        with ThreadPoolExecutor(max_workers=5) as executor:
            futures = {executor.submit(self.discover_books_in_category, cat): cat 
                      for cat in categories}
            
            for future in tqdm(as_completed(futures), total=len(categories), desc="Scanning"):
                category = futures[future]
                try:
                    books = future.result()
                    all_books.extend(books)
                    if books:
                        tqdm.write(f"  {category}: {len(books)} books")
                except Exception as e:
                    tqdm.write(f"  {category}: Error - {e}")
        
        self.stats['discovered'] = len(all_books)
        print(f"\n✓ Discovered {len(all_books)} total books")
        return all_books
    
    def _clean_title(self, title: str) -> str:
        """Clean up book title."""
        title = re.sub(r"^\d+[\._-]?\s*", "", title)
        title = re.sub(r"[\._-]+", " ", title)
        title = re.sub(r"\s+", " ", title)
        return title.strip()[:200]
    
    def _get_safe_filename(self, book: BookInfo) -> str:
        """Create safe filename."""
        safe_category = re.sub(r'[<>:"/\\|?*&]', "_", book.category)
        safe_title = re.sub(r'[<>:"/\\|?*&]', "_", book.title)
        if len(safe_title) > 150:
            safe_title = safe_title[:150]
        return f"[{safe_category}] {safe_title}.pdf"
    
    def _download_single(self, book: BookInfo) -> Tuple[BookInfo, bool, str]:
        """Download a single book (called by thread pool)."""
        filename = self._get_safe_filename(book)
        filepath = self.output_dir / filename
        
        # Skip if exists
        if filepath.exists() and filepath.stat().st_size > 1000:
            return book, True, "exists"
        
        try:
            response = self._get_session().get(
                book.url,
                timeout=self.timeout,
                stream=True
            )
            response.raise_for_status()
            
            # Verify it's a PDF
            content_type = response.headers.get("Content-Type", "")
            if "html" in content_type.lower():
                return book, False, "not_pdf"
            
            # Download
            with open(filepath, "wb") as f:
                for chunk in response.iter_content(chunk_size=32768):  # Larger chunks
                    f.write(chunk)
            
            return book, True, "downloaded"
            
        except Exception as e:
            return book, False, str(e)[:50]
    
    def download_all_parallel(self, books: List[BookInfo]) -> Dict[str, int]:
        """
        Download all books in parallel.
        
        Args:
            books: List of BookInfo objects
            
        Returns:
            Statistics dictionary
        """
        print(f"\n⚡ Downloading {len(books)} books with {self.max_workers} parallel connections...\n")
        
        with ThreadPoolExecutor(max_workers=self.max_workers) as executor:
            # Submit all download tasks
            futures = {executor.submit(self._download_single, book): book for book in books}
            
            # Process as they complete
            for future in tqdm(as_completed(futures), total=len(books), desc="Downloading"):
                try:
                    book, success, status = future.result()
                    
                    with self._lock:
                        if success:
                            if status == "exists":
                                self.stats['already_exists'] += 1
                            else:
                                self.stats['downloaded'] += 1
                        else:
                            self.stats['failed'] += 1
                            if "not_pdf" not in status:
                                tqdm.write(f"  ✗ {book.title[:40]}... - {status}")
                                
                except Exception as e:
                    with self._lock:
                        self.stats['failed'] += 1
        
        return self.stats
    
    def run(self, categories: List[str] = None) -> Dict[str, int]:
        """Run full scraping pipeline."""
        start_time = time.time()
        
        # Discover books
        books = self.discover_all_books(categories)
        
        if not books:
            print("No books found")
            return self.stats
        
        # Download in parallel
        self.download_all_parallel(books)
        
        elapsed = time.time() - start_time
        
        # Summary
        print("\n" + "="*50)
        print("DOWNLOAD SUMMARY")
        print("="*50)
        print(f"Discovered:      {self.stats['discovered']}")
        print(f"Downloaded:      {self.stats['downloaded']}")
        print(f"Already existed: {self.stats['already_exists']}")
        print(f"Failed:          {self.stats['failed']}")
        print(f"Time elapsed:    {elapsed/60:.1f} minutes")
        print(f"Speed:           {len(books)/elapsed:.1f} books/second")
        print("="*50)
        
        return self.stats

## 3. Run the Fast Scraper

In [ ]:
# Initialize FAST scraper with 10 parallel downloads
scraper = FastDentalLibraryScraper(
    output_dir=DRIVE_FOLDER,
    max_workers=10,  # 10 parallel downloads (adjust if needed)
    timeout=60
)

print(f"Output folder: {DRIVE_FOLDER}")
print(f"Parallel downloads: {scraper.max_workers}")
print(f"Expected speedup: ~{scraper.max_workers}x faster")
print()

In [ ]:
# Run the scraper!
stats = scraper.run()

In [ ]:
# Alternative: Scrape specific categories only
# selected_categories = [
#     "Conservative Dentistry",
#     "Orthodontics",
#     "Prosthodontics",
#     "Periodontics",
# ]
# stats = scraper.run(categories=selected_categories)

## 4. Verify Downloaded Books

In [ ]:
# Count all PDFs
all_pdfs = [f for f in os.listdir(DRIVE_FOLDER) if f.lower().endswith('.pdf')]
print(f"Total dental books in Google Drive: {len(all_pdfs)}")

# Total size
total_size = sum(os.path.getsize(os.path.join(DRIVE_FOLDER, f)) for f in all_pdfs)
print(f"Total size: {total_size / (1024**3):.2f} GB")

# Count by category
print("\nBooks by category:")
category_counts = {}
for pdf in all_pdfs:
    match = re.match(r'\[([^\]]+)\]', pdf)
    cat = match.group(1) if match else 'Uncategorized'
    category_counts[cat] = category_counts.get(cat, 0) + 1

for cat, count in sorted(category_counts.items(), key=lambda x: -x[1]):
    print(f"  {cat}: {count}")

## 5. Extract Text from All PDFs

In [ ]:
import fitz  # PyMuPDF
import json
from concurrent.futures import ProcessPoolExecutor
import multiprocessing

def extract_pdf_text(pdf_path: str) -> list:
    """Extract text from a PDF file."""
    try:
        doc = fitz.open(pdf_path)
        pages = []
        filename = os.path.basename(pdf_path)
        
        # Extract category
        match = re.match(r'\[([^\]]+)\]', filename)
        category = match.group(1) if match else "Unknown"
        
        for page_num in range(len(doc)):
            page = doc[page_num]
            text = page.get_text("text")
            text = " ".join(text.split())
            
            if len(text) >= 100:
                pages.append({
                    "text": text,
                    "source": filename,
                    "category": category,
                    "page_number": page_num + 1,
                    "total_pages": len(doc)
                })
        
        doc.close()
        return pages
    except Exception as e:
        return []

In [ ]:
# Output directory
OUTPUT_DIR = "/content/drive/MyDrive/RAFT_dental_data"
os.makedirs(OUTPUT_DIR, exist_ok=True)

output_file = os.path.join(OUTPUT_DIR, "raw_pages.jsonl")

print(f"Extracting text from {len(all_pdfs)} PDFs...")
print(f"Output: {output_file}")
print()

total_pages = 0
processed = 0
failed = 0

with open(output_file, "w", encoding="utf-8") as f:
    for pdf_name in tqdm(all_pdfs, desc="Extracting"):
        pdf_path = os.path.join(DRIVE_FOLDER, pdf_name)
        pages = extract_pdf_text(pdf_path)
        
        if pages:
            for page in pages:
                f.write(json.dumps(page, ensure_ascii=False) + "\n")
                total_pages += 1
            processed += 1
        else:
            failed += 1

print(f"\n✓ Done! Processed: {processed}, Failed: {failed}, Total pages: {total_pages}")

## 6. Chunk Text for Embedding

In [ ]:
!pip install -q langchain-text-splitters tiktoken

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
import tiktoken

tokenizer = tiktoken.get_encoding("cl100k_base")
def count_tokens(text): return len(tokenizer.encode(text))

splitter = RecursiveCharacterTextSplitter(
    chunk_size=2048, chunk_overlap=400,
    length_function=count_tokens,
    separators=["\n\n", "\n", ". ", " ", ""]
)

print("Chunker ready: ~512 tokens per chunk, 100 token overlap")

In [ ]:
chunks_file = os.path.join(OUTPUT_DIR, "chunks.jsonl")

total_chunks = 0
with open(output_file, "r", encoding="utf-8") as fin, \
     open(chunks_file, "w", encoding="utf-8") as fout:
    
    for line in tqdm(fin, desc="Chunking"):
        page = json.loads(line)
        if len(page["text"]) < 100:
            continue
        
        chunks = splitter.split_text(page["text"])
        
        for idx, chunk_text in enumerate(chunks):
            chunk = {
                "chunk_id": f"{page['source']}_{page['page_number']}_{idx}",
                "text": chunk_text,
                "source": page["source"],
                "category": page.get("category", "Unknown"),
                "page_number": page["page_number"],
                "chunk_index": idx,
                "token_count": count_tokens(chunk_text)
            }
            fout.write(json.dumps(chunk, ensure_ascii=False) + "\n")
            total_chunks += 1

print(f"\n✓ Created {total_chunks} chunks")

## 7. Summary

In [ ]:
raw_size = os.path.getsize(output_file) / (1024**2) if os.path.exists(output_file) else 0
chunks_size = os.path.getsize(chunks_file) / (1024**2) if os.path.exists(chunks_file) else 0

print("="*60)
print("FINAL SUMMARY")
print("="*60)
print(f"📁 Books: {len(all_pdfs)} PDFs ({total_size/(1024**3):.2f} GB)")
print(f"📄 Pages: {total_pages} ({raw_size:.1f} MB)")
print(f"🧩 Chunks: {total_chunks} ({chunks_size:.1f} MB)")
print(f"📂 Output: {OUTPUT_DIR}")
print("="*60)
print("Next: Run 02_qa_generation.ipynb")
print("="*60)